# Install conda on your Colab environment

Ignore this first cell if you are running the notebook in a local environment.

One can still run it locally but it will have no effect.

In [1]:
# Run this cell first - it will install a conda distribution (mamba)
# on your Drive then restart the kernel automatically 
# (don't worry about the crashing/restarting kernel messages)
# It HAS to be runned FIRST everytime you use the notebook in colab

import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

# Set up your Colab or local environment
# Then import libraries

Run this cell in both cases of use (local or Colab)

In [2]:
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    
    # Check everything is fine with conda in Colab
    import condacolab
    condacolab.check()
    
    # Mount your drive environment in the colab runtime
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    
    # Change this variable to your path on Google Drive to which the repo has been cloned
    # If you followed the colab notebook 'repo_cloning.ipynb', nothing to change here
    repo_path_in_drive = '/content/drive/My Drive/Github/amn_release/'
    # Change directory to your repo cloned in your drive
    DIRECTORY = repo_path_in_drive
    os.chdir(repo_path_in_drive)
    # Copy the environment given in the environment_amn_light.yml
    !mamba env update -n base -f environment_amn_light.yml
    
    # This is one of the few Colab-compatible font
    font = 'Liberation Sans'
    
else:
    
    # In this case the local root of the repo is our working directory
    DIRECTORY = './'
    font = 'arial'

# printing the working directory files. One can check you see the same folders and files as in the git webpage.
print(os.listdir(DIRECTORY))

from Library.Build_Model import *

# We declare this function here and not in the
# function-storing python file to modify it easily
# as it can change the printouts of the methods
def printout(filename, Stats, model, time): 
    # printing Stats
    print('Stats for %s CPU-time %.4f' % (filename, time))
    print('R2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    print('Q2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.test_objective[0], Stats.test_objective[1],
           Stats.test_loss[0], Stats.test_loss[1]))

['Build_Model_RC.ipynb', 'Duplicate_Model.ipynb', 'Dataset_experimental', 'Result', '.DS_Store', 'Dataset_model', 'LICENSE', 'Reservoir', 'Build_Model_MM.ipynb', 'Build_Experimental.ipynb', 'Build_Model_ANN_Dense.ipynb', 'Build_Model_AMN.ipynb', 'environment_amn.yml', 'Library', 'README.md', '.gitignore', 'Figures', 'Build_Dataset.ipynb', '.ipynb_checkpoints', 'Dataset_input', 'Figures.ipynb', 'Tutorial.ipynb', 'environment_amn_light.yml']


# Reservoir computing with experimental data set

This cell loads a reservoir (an AMN trained on *in silico* data with its parameters fixed), then learns on an experimental dataset.

The creation of both *in silico* and experimental datasets is done separately, in `Build_Dataset.ipynb`.

In [ ]:
# Create, train and evaluate RC E. coli models on experimental training set 
# Repeat the process with different seeds
# This cell takes several hours to execute

seed = 10
Maxloop, Q2, PRED = 3, [], []  
for Nloop in range(Maxloop):
    # What you can change 
    seed = Nloop
    np.random.seed(seed=seed)  
    trainname = 'iML1515_EXP'
    reservoirname = 'iML1515_UB_AMN_QP'
    # End of What you can change
  
    # Create model 
    trainingfile = DIRECTORY+'Dataset_input/'+trainname
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    X, Y = read_XY(trainingfile, 38)
    model = RC_Model(reservoirfile = reservoirfile, X=X, Y=Y, 
                n_hidden_prior = 1, hidden_dim_prior = 500, activation_prior = 'relu',
                epochs = 1000, train_rate = 1.0e-4, xfold = 0,
                verbose=True)
    model.printout()

    # Train and evaluate
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=True)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred)

# Some printings and savings
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print(PRED.shape)
obj = PRED[:,:,0]
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_'\
+str(model.model_type)+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_'\
+str(model.model_type)+'_PRED.csv'
np.savetxt(filename, obj, delimiter=',')

# Saving input / output of the first prediction for Cobra runs
pred, obj = PRED[0], pred[:,0]
Loss_SV, Loss_Vin, Loss_Vpos = pred[:,1], pred[:,2], pred[:,3]
print('Loss SV average %.4f max %.4f' % (np.mean(Loss_SV), np.max(Loss_SV)))
print('Loss Vin average %.4f max %.4f' % (np.mean(Loss_Vin), np.max(Loss_Vin)))
print('Loss Vpos average %.4f max %.4f' % (np.mean(Loss_Vpos), np.max(Loss_Vpos)))
V = pred[:, 4:4+model.S.shape[1]]
Vin = pred[:, 4+model.S.shape[1]:]
Vin = model.res.scaler * Vin # rescale back for Cobra
XY = np.concatenate((Vin, Y), axis=1)
filename = DIRECTORY+'Result/'+reservoirname+'_'\
+str(model.model_type)+'_solution_for_Cobra.csv'
np.savetxt(filename, XY, delimiter=',')

In [20]:
# Train a P. Putida RC models on FBA simulated data
# Test RC with experimental data

# What you can change
seed = 10
np.random.seed(seed=seed)  
trainname = 'IJN1463_SIMULATED_EXP' # IJN1463_SIMULATED_EXP for test IJN1463_EXP for train
testname = 'IJN1463_EXP'
reservoirname = 'IJN1463_10_UB_AMN_QP'
# End of What you can change

# Create model 
trainingfile   = DIRECTORY+'Dataset_input/'+trainname
testfile       = DIRECTORY+'Dataset_input/'+testname
reservoirfile  = DIRECTORY+'Reservoir/'+reservoirname    

Nloop = 10
for l in range(Nloop):
    Xtrain, Ytrain = read_XY(trainingfile, 196)
    Xtest, Ytest   = read_XY(testfile, 196)    
    model = RC_Model(reservoirfile = reservoirfile, X=Xtrain, Y=Ytrain, 
                 n_hidden_prior = 1, hidden_dim_prior = 1000, activation_prior = 'relu',
                 epochs = 500, train_rate = 1.0e-3, xfold = 0,
                 n_hidden_post = 0, activation_post = 'sigmoid',
                 regression = False,
                 verbose = False)
    #model.printout()
        
    # Train 
    start_time = time.time()
    try:
        reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
    except:
        continue
    delta_time = time.time() - start_time
    
    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    acc = accuracy_score(Ytrain, np.rint(pred[:,0]))
    print('%d Acc train = %.4f' % (l, acc))
    
    # Test with experimental dataset
    pred, stats = evaluate_model(reservoir.model, Xtest, Ytest, 
                                 model, inputmodel=True, verbose=False)
    acc = accuracy_score(Ytest, np.rint(pred[:,0]))
    print('%d Acc test = %.4f' % (l, acc))

    # Saving test input / output of the first prediction for Cobra runs
    Loss_SV, Loss_Vin, Loss_Vpos = pred[:,1], pred[:,2], pred[:,3]
    #print('Loss SV average %.4f max %.4f' % (np.mean(Loss_SV), np.max(Loss_SV)))
    #print('Loss Vin average %.4f max %.4f' % (np.mean(Loss_Vin), np.max(Loss_Vin)))
    #print('Loss Vpos average %.4f max %.4f' % (np.mean(Loss_Vpos), np.max(Loss_Vpos)))
    V   = pred[:, 4:4+model.S.shape[1]]
    Vin = pred[:, 4+model.S.shape[1]:]
    Vin = model.res.scaler * Vin # rescale back for Cobra
    XY = np.concatenate((Vin, Ytest), axis=1)
    filename = DIRECTORY+'Dataset_input/'+reservoirname\
    +'_for_Cobra_test_'+str(l)+'.csv'
    np.savetxt(filename, XY, delimiter=',')

nbr parameters: 1517311
nbr parameters: 1517311
8/8 [==============================] - 0s 5ms/step
---------- 1
8/8 [==============================] - 0s 5ms/step
---------- 1
train = 0.95 test = 0.95 loss-train = -1.000000 loss-test = -1.000000 iter=0
Stats for IJN1463_10_UB_AMN_QP CPU-time 271.1401
R2 = 0.9547 (+/- 0.0000) Constraint = -1.0000 (+/- 0.0000)
Q2 = 0.9547 (+/- 0.0000) Constraint = -1.0000 (+/- 0.0000)
1 Acc train = 0.9547
8/8 [==============================] - 0s 5ms/step
---------- 1
1 Acc test = 0.7407
nbr parameters: 1517311
8/8 [==============================] - 0s 5ms/step
---------- 1
8/8 [==============================] - 0s 4ms/step
---------- 1
train = 0.95 test = 0.95 loss-train = -1.000000 loss-test = -1.000000 iter=0
Stats for IJN1463_10_UB_AMN_QP CPU-time 275.9248
R2 = 0.9547 (+/- 0.0000) Constraint = -1.0000 (+/- 0.0000)
Q2 = 0.9547 (+/- 0.0000) Constraint = -1.0000 (+/- 0.0000)
2 Acc train = 0.9547
8/8 [==============================] - 0s 4ms/step
-------